# MPIIGaze Dataset Evaluation

This notebook evaluates the existing eye-tracking model on the MPIIGaze dataset with error output in degrees of visual angle for direct comparison with research literature.

## Dataset Overview
- **MPIIGaze**: 213,659 images from 15 participants during natural laptop use
- **Evaluation**: Leave-one-person-out cross-validation
- **Metric**: Mean angular error in degrees
- **Benchmarks**: State-of-the-art ranges from 4.3° to 10.8° depending on evaluation protocol

## Setup and Dependencies

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import keras.ops as ops
from scipy.io import loadmat
import urllib.request
import tarfile
import json
from pathlib import Path
import cv2
from sklearn.model_selection import LeaveOneOut
import pickle

## 1. Download and Extract MPIIGaze Dataset

In [2]:
# Create data directory
data_dir = Path("./mpiigaze_data")
data_dir.mkdir(exist_ok=True)

# Download URL
dataset_url = "http://datasets.d2.mpi-inf.mpg.de/MPIIGaze/MPIIGaze.tar.gz"
dataset_file = data_dir / "MPIIGaze.tar.gz"
extract_dir = data_dir / "MPIIGaze"

In [3]:
# Download the dataset if not already present
if not dataset_file.exists():
    print("Downloading MPIIGaze dataset...")
    urllib.request.urlretrieve(dataset_url, dataset_file)
    print(f"Download complete: {dataset_file}")
else:
    print(f"Dataset already downloaded: {dataset_file}")

# Extract the dataset if not already extracted
if not extract_dir.exists():
    print("Extracting dataset...")
    with tarfile.open(dataset_file, 'r:gz') as tar:
        tar.extractall(data_dir)
    print(f"Extraction complete: {extract_dir}")
else:
    print(f"Dataset already extracted: {extract_dir}")

Download complete: mpiigaze_data/MPIIGaze.tar.gz
Extracting dataset...
Extraction complete: mpiigaze_data/MPIIGaze


In [4]:
# Define the path to the evaluation subset annotation files
# Assuming data_dir is defined in a previous cell
evaluation_dir = data_dir / "MPIIGaze" / "Evaluation Subset"
annotation_dir = evaluation_dir / "annotation for face image"

# Get the list of subject IDs from the annotation files
subject_ids = []
if annotation_dir.exists():
    # List files in the annotation directory that match the pattern p*.txt
    annotation_files = list(annotation_dir.glob("p*.txt"))
    # Extract subject IDs (e.g., 'p00' from 'p00.txt') and sort them
    subject_ids = sorted([f.stem for f in annotation_files])
    print(f"Found {len(subject_ids)} subject IDs: {subject_ids}")
else:
    print(f"Annotation directory not found: {annotation_dir}")

# Store the number of subjects for later use
num_subjects = len(subject_ids)
print(f"Number of subjects for evaluation: {num_subjects}")

Found 15 subject IDs: ['p00', 'p01', 'p02', 'p03', 'p04', 'p05', 'p06', 'p07', 'p08', 'p09', 'p10', 'p11', 'p12', 'p13', 'p14']
Number of subjects for evaluation: 15


In [5]:
def load_images_from_paths(image_data):
    """
    Load images from a list of (path, eye_type) tuples.

    Args:
        image_data (list): A list of tuples, where each tuple is (full_image_path, eye_type).

    Returns:
        list: A list of loaded images (as numpy arrays) and their corresponding eye types.
    """
    loaded_images = []
    print(f"Attempting to load {len(image_data)} images...")
    for i, (img_path, eye_type) in enumerate(image_data):
        if i % 100 == 0:
            print(f"  Loading image {i}/{len(image_data)}")
        try:
            # Using cv2.IMREAD_GRAYSCALE as images are grayscale
            img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                loaded_images.append((img, eye_type))
            else:
                print(f"Warning: Could not load image at {img_path}")
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")

    print(f"Successfully loaded {len(loaded_images)} images.")
    return loaded_images

In [6]:
def crop_black_padding(image):
    """
    Crops an image by removing black padding and adjusting to 36x144 aspect ratio.
    Prioritizes removing black pixels over preserving image content.

    Args:
        image (np.ndarray): A grayscale image (NumPy array).

    Returns:
        np.ndarray: The cropped image with 36x144 aspect ratio.
    """
    # Find the coordinates of all non-zero pixels
    non_zero_coords = np.nonzero(image)

    # If no non-zero pixels are found, return a small black image
    if non_zero_coords[0].size == 0:
        return np.zeros((36, 144), dtype=image.dtype)

    # Get bounding box of non-zero pixels
    min_row, max_row = np.min(non_zero_coords[0]), np.max(non_zero_coords[0])
    min_col, max_col = np.min(non_zero_coords[1]), np.max(non_zero_coords[1])

    # Calculate current dimensions
    height = max_row - min_row + 1
    width = max_col - min_col + 1

    # Target aspect ratio is 144/36 = 4.0 (width/height)
    target_aspect_ratio = 4.0

    # Calculate center of bounding box
    center_row = (min_row + max_row) // 2
    center_col = (min_col + max_col) // 2

    # Determine final dimensions based on aspect ratio
    if width / height > target_aspect_ratio:
        # Width is limiting factor - use full width, adjust height
        final_width = width
        final_height = int(width / target_aspect_ratio)
    else:
        # Height is limiting factor - use full height, adjust width
        final_height = height
        final_width = int(height * target_aspect_ratio)

    # Calculate crop boundaries centered on bounding box
    crop_min_row = center_row - final_height // 2
    crop_max_row = crop_min_row + final_height - 1
    crop_min_col = center_col - final_width // 2
    crop_max_col = crop_min_col + final_width - 1

    # Ensure boundaries are within image bounds
    crop_min_row = max(0, crop_min_row)
    crop_max_row = min(image.shape[0] - 1, crop_max_row)
    crop_min_col = max(0, crop_min_col)
    crop_max_col = min(image.shape[1] - 1, crop_max_col)

    # Extract the cropped region
    cropped_image = image[crop_min_row:crop_max_row+1, crop_min_col:crop_max_col+1]

    return cropped_image

print("Simplified crop_black_padding function - prioritizes black pixel removal with 36x144 aspect ratio.")

Simplified crop_black_padding function - prioritizes black pixel removal with 36x144 aspect ratio.


## Consolidate data loading and preprocessing

### Subtask:
Create a single function or code block that loads the image paths for a subject, crops the images to the desired aspect ratio, and then resizes them to 36x144.


In [7]:
def load_all_subject_data(subject_id):
    """
    Loads all image paths for a subject from both evaluation and calibration sets.
    Marks evaluation images (from annotation files) vs calibration images.

    Args:
        subject_id (str): The ID of the subject (e.g., 'p00').

    Returns:
        list: A list of tuples (full_image_path, eye_type, image_index, is_evaluation, day).
              Returns None if subject data cannot be loaded.
    """
    base_data_dir = Path("./mpiigaze_data/MPIIGaze")
    evaluation_dir = base_data_dir / "Evaluation Subset"
    data_dir = base_data_dir / "Data" / "Original" / subject_id

    if not data_dir.exists():
        print(f"Subject data directory not found: {data_dir}")
        return None

    # Load evaluation image paths from annotation file
    evaluation_annotation_file = evaluation_dir / "sample list for eye image" / f"{subject_id}.txt"
    evaluation_images = set()

    if evaluation_annotation_file.exists():
        with open(evaluation_annotation_file, 'r') as f:
            for line in f:
                parts = line.strip().split(' ')
                if len(parts) >= 2:
                    relative_path = parts[0].replace('\\', '/')
                    evaluation_images.add(relative_path)

    # Scan all image directories for the subject
    all_image_data = []

    # Look for day directories (like day01, day02, etc.)
    for day_dir in sorted(data_dir.iterdir()):
        if not day_dir.is_dir() or day_dir.name == "Calibration":
            continue

        day_name = day_dir.name

        # Look for annotation.txt to determine available images
        annotation_file = day_dir / "annotation.txt"
        if not annotation_file.exists():
            continue

        # Read annotation file to get image count and data
        with open(annotation_file, 'r') as f:
            annotation_lines = f.readlines()

        # Process each image in the day directory
        for image_index, annotation_line in enumerate(annotation_lines):
            # Construct image filename (assuming 4-digit zero-padded)
            image_filename = f"{image_index + 1:04d}.jpg"
            relative_path = f"{day_name}/{image_filename}"
            full_image_path = day_dir / image_filename

            if not full_image_path.exists():
                continue

            # Determine if this is an evaluation image
            is_evaluation = relative_path in evaluation_images

            # For simplicity, assume all images are left eye (can be refined later)
            eye_type = "left"  # This could be extracted from evaluation annotation if needed

            all_image_data.append((
                full_image_path,
                eye_type,
                image_index,
                is_evaluation,
                day_name
            ))

    print(f"Loaded {len(all_image_data)} total images for subject {subject_id}")
    evaluation_count = sum(1 for _, _, _, is_eval, _ in all_image_data if is_eval)
    calibration_count = len(all_image_data) - evaluation_count
    print(f"  - {evaluation_count} evaluation images")
    print(f"  - {calibration_count} calibration images")

    return all_image_data

print("Created unified load_all_subject_data function for evaluation and calibration images.")

Created unified load_all_subject_data function for evaluation and calibration images.


In [8]:
from scipy.io import loadmat
from pathlib import Path
import numpy as np # Import numpy for array handling

# Assuming subject_ids is available from a previous cell
if 'subject_ids' not in locals() or not subject_ids:
    print("Subject IDs not found. Please run the cell to load subject IDs first.")
else:
    # Define the base path to the data directory
    base_data_dir = Path("./mpiigaze_data/MPIIGaze/Data/Original")

    # Initialize an empty dictionary to store screen sizes
    screen_sizes_lookup = {}

    print("Loading screen sizes for all subjects...")

    # Iterate through each subject ID
    for subject_id in subject_ids:
        # Construct the path to the screenSize.mat file for the current subject
        mat_file_path = base_data_dir / subject_id / "Calibration" / "screenSize.mat"

        # Check if the file exists
        if mat_file_path.exists():
            try:
                # Load the .mat file
                mat_data = loadmat(mat_file_path)

                # Extract width_pixel and height_pixel
                # The data is likely stored as NumPy arrays, extract the scalar value
                width_pixel = int(mat_data['width_pixel'][0][0])
                height_pixel = int(mat_data['height_pixel'][0][0])

                # Store in the dictionary as a tuple
                screen_sizes_lookup[subject_id] = (width_pixel, height_pixel)
                # print(f"Loaded screen size for {subject_id}: ({width_pixel}, {height_pixel})") # Optional: print each loaded size

            except Exception as e:
                print(f"Error loading or parsing screenSize.mat for {subject_id}: {e}")
        else:
            print(f"Warning: screenSize.mat not found for {subject_id} at: {mat_file_path}")

    print(screen_sizes_lookup)

Loading screen sizes for all subjects...
{'p00': (1280, 800), 'p01': (1440, 900), 'p02': (1280, 800), 'p03': (1440, 900), 'p04': (1280, 800), 'p05': (1440, 900), 'p06': (1680, 1050), 'p07': (1440, 900), 'p08': (1440, 900), 'p09': (1440, 900), 'p10': (1440, 900), 'p11': (1280, 800), 'p12': (1280, 800), 'p13': (1280, 800), 'p14': (1440, 900)}


In [9]:
def load_gaze_annotation(image_path, image_index, screen_size):
    """
    Loads the on-screen gaze target position from the annotation.txt file
    associated with a given image path, using the image index to find the correct line.
    Converts pixel coordinates to proportions of screen size using a lookup dictionary.

    Args:
        image_path (Path): The full path to the image file.
        image_index (int): The 0-based index of the image within its directory's sorted file list.
        subject_id (str): The ID of the subject (e.g., 'p00').
        screen_sizes_lookup (dict): A dictionary with subject IDs as keys and (width_pixel, height_pixel) tuples as values.


    Returns:
        np.ndarray: The on-screen gaze target position as proportions (between 0 and 1)
                    as a numpy array, or None if the annotation is not found, cannot be parsed,
                    or screen size is not available.
    """
    # The annotation file is in the same directory as the image
    annotation_file = image_path.parent / "annotation.txt"

    if not annotation_file.exists():
        # print(f"Annotation file not found for image: {image_path}") # Keep this quiet for batch processing
        return None


    screen_width, screen_height = screen_size

    try:
        with open(annotation_file, 'r') as f:
            # Read all lines and get the one corresponding to the image index
            annotation_lines = f.readlines()

            if image_index < 0 or image_index >= len(annotation_lines):
                print(f"Warning: Image index {image_index} is out of bounds for annotation file {annotation_file} with {len(annotation_lines)} lines. Skipping.")
                return None

            # Get the specific annotation line
            line = annotation_lines[image_index].strip()
            parts = line.split(' ')

            # Assuming the lines are JUST the annotation data (no filename at the start)
            # Extract Dimensions 25 and 26 (0-indexed, so indices 24 and 25 in parts)
            if len(parts) >= 26:
                try:
                    gaze_x_pixel = float(parts[24])
                    gaze_y_pixel = float(parts[25])

                    # Convert pixel coordinates to proportions
                    gaze_x_prop = gaze_x_pixel / screen_width
                    gaze_y_prop = gaze_y_pixel / screen_height

                    # Ensure proportions are within [0, 1] range (optional, depending on expected data)
                    # gaze_x_prop = np.clip(gaze_x_prop, 0.0, 1.0)
                    # gaze_y_prop = np.clip(gaze_y_prop, 0.0, 1.0)


                    return np.array([gaze_x_prop, gaze_y_prop], dtype=np.float32)

                except ValueError:
                    print(f"Could not parse gaze coordinates in line {image_index} of {annotation_file}: {line}. Skipping.")
                    return None
            else:
                # print(f"Annotation line {image_index} is too short in {annotation_file}: {line}. Skipping.") # Keep quiet for batch
                return None
    except Exception as e:
        print(f"Error reading annotation file {annotation_file}: {e}")
        return None

print("load_gaze_annotation function modified to convert gaze coordinates to proportions.")

load_gaze_annotation function modified to convert gaze coordinates to proportions.


In [10]:
import random # Import the random module

def preprocess_image(image_path):
    """
    Loads and preprocesses a single image: reads in grayscale, crops black padding,
    and resizes to 36x144 pixels.

    Args:
        image_path (Path): Path to the image file.

    Returns:
        np.ndarray or None: Preprocessed image (36, 144) or None if processing failed.
    """
    try:
        # Read image in grayscale
        img = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
        if img is None:
            return None

        # Crop black padding and adjust aspect ratio
        cropped_img = crop_black_padding(img)

        # Resize to target dimensions (width=144, height=36)
        resized_img = cv2.resize(cropped_img, (144, 36))

        return resized_img

    except Exception as e:
        print(f"Error preprocessing image {image_path}: {e}")
        return None

def process_subject_data(subject_id):
    """
    Processes images for a subject, selecting only evaluation images.

    Args:
        subject_id (str): The ID of the subject (e.g., 'p00').

    Returns:
        list: List of tuples (preprocessed_image, eye_type, gaze_coords, is_evaluation, day)
              containing only evaluation images, or None if processing failed.
    """
    # Load all image data for the subject (paths and metadata)
    all_image_data = load_all_subject_data(subject_id)
    if all_image_data is None:
        return None

    # Filter to include only evaluation images
    evaluation_data = [item for item in all_image_data if item[3]] # item[3] is is_evaluation

    print(f"Subject {subject_id}: Found {len(evaluation_data)} evaluation images.")


    data_to_process = evaluation_data

    print(f"Subject {subject_id}: Total images to process after filtering: {len(data_to_process)}")


    # Get screen size for gaze coordinate conversion
    screen_size = screen_sizes_lookup.get(subject_id)
    if screen_size is None:
        print(f"Screen size not found for subject {subject_id}")
        return None

    processed_data = []
    print(f"Processing {len(data_to_process)} images for subject {subject_id}...")

    for i, (img_path, eye_type, image_index, is_evaluation, day) in enumerate(data_to_process):
        if (i + 1) % 100 == 0:
            print(f"  Processed {i+1}/{len(data_to_process)} images")

        # Preprocess the image
        preprocessed_img = preprocess_image(img_path)
        if preprocessed_img is None:
            continue

        # Load gaze annotation
        # Pass subject_id and screen_sizes_lookup to load_gaze_annotation
        gaze_coords = load_gaze_annotation(img_path, image_index, screen_size)
        if gaze_coords is None:
            continue

        processed_data.append((
            preprocessed_img,
            eye_type,
            gaze_coords,
            is_evaluation,
            day
        ))

    print(f"Successfully processed {len(processed_data)} images for subject {subject_id}")
    return processed_data

print("Updated processing functions to filter for only evaluation images.")

Updated processing functions to filter for only evaluation images.


# Task
Convert the preprocessed MPIIGaze dataset, including cropped eye images, gaze coordinates (as proportions of screen size), day, and subject ID, into a TFRecords dataset that can be parsed by the provided `parse` function (with 'phase' replaced by 'day').

## Define tfrecord features

### Subtask:
Define the structure of the features that will be stored in the TFRecords, including 'day', 'subject_id', 'eye_img', 'x', and 'y'.


**Reasoning**:
Define the feature structure for the TFRecords dataset as specified in the instructions.



In [11]:
# Define the feature structure for the TFRecords
feature_description = {
    'day': tf.io.FixedLenFeature([], tf.string),
    'subject_id': tf.io.FixedLenFeature([], tf.string),
    'eye_img': tf.io.FixedLenFeature([], tf.string),  # Image data stored as a serialized byte string
    'x': tf.io.FixedLenFeature([], tf.float32),      # Gaze x-coordinate (proportion)
    'y': tf.io.FixedLenFeature([], tf.float32),      # Gaze y-coordinate (proportion)
    # Removed 'is_evaluation' feature as we are only processing evaluation images
}

print("Updated TFRecords feature description by removing is_evaluation.")
print(feature_description)

Updated TFRecords feature description by removing is_evaluation.
{'day': FixedLenFeature(shape=[], dtype=tf.string, default_value=None), 'subject_id': FixedLenFeature(shape=[], dtype=tf.string, default_value=None), 'eye_img': FixedLenFeature(shape=[], dtype=tf.string, default_value=None), 'x': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None), 'y': FixedLenFeature(shape=[], dtype=tf.float32, default_value=None)}


## Create a serialization function

### Subtask:
Write a function that takes a preprocessed data entry (image, eye type, gaze coordinates) and the corresponding day and subject ID, and serializes it into a TFRecord `Example` protocol buffer.


**Reasoning**:
The subtask is to write a function that serializes a single data entry into a TFRecord Example protocol buffer. This involves converting data types and creating a tf.train.Example object.



In [12]:
def serialize_example(image, eye_type, gaze_coords, is_evaluation, day, subject_id):
    """
    Serializes a single data entry into a TFRecord Example protocol buffer.
    is_evaluation is included in the input for compatibility but not serialized.

    Args:
        image (np.ndarray): The preprocessed eye image (36x144, grayscale).
        eye_type (str): The type of eye ('left' or 'right').
        gaze_coords (np.ndarray): The gaze coordinates as proportions [x, y].
        is_evaluation (bool): True for evaluation images, False for calibration (not serialized).
        day (str): The day the data was recorded.
        subject_id (str): The ID of the subject.

    Returns:
        bytes: The serialized tf.train.Example protocol buffer.
    """
    # Convert image to byte string
    image_bytes = image.tobytes()

    # Convert strings to byte strings
    day_bytes = day.encode('utf-8')
    subject_id_bytes = subject_id.encode('utf-8')

    # Create feature dictionary (excluding is_evaluation)
    feature = {
        'day': tf.train.Feature(bytes_list=tf.train.BytesList(value=[day_bytes])),
        'subject_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[subject_id_bytes])),
        'eye_img': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_bytes])),
        'x': tf.train.Feature(float_list=tf.train.FloatList(value=[gaze_coords[0]])),
        'y': tf.train.Feature(float_list=tf.train.FloatList(value=[gaze_coords[1]])),
        # Removed 'is_evaluation' feature from serialization
    }

    # Create and serialize Example
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

print("Updated serialize_example function by removing is_evaluation feature.")

Updated serialize_example function by removing is_evaluation feature.


## Iterate through subjects and days

### Subtask:
Loop through each subject and then through each day's data for that subject to prepare for data loading and preprocessing.


**Reasoning**:
Iterate through each subject and day to prepare for loading and preprocessing data for TFRecords creation.



**Reasoning**:
The loops for iterating through subjects and days have been successfully set up, and the data for each subject is grouped by day. The next logical step within this structure is to load the image data, preprocess it, get the gaze annotations, and then serialize this data into TFRecords for each day.



In [13]:
# Create simplified TFRecords processing pipeline
tfrecords_output_dir = Path("./mpiigaze_tfrecords_simplified")
tfrecords_output_dir.mkdir(exist_ok=True)
print(f"TFRecords output directory: {tfrecords_output_dir}")

def create_tfrecords_for_all_subjects():
    """
    Creates one TFRecord file per subject containing all their images
    (both evaluation and calibration) with proper labeling.
    """
    print("Starting simplified TFRecords creation for all subjects...")

    for subject_id in subject_ids:
        print(f"\n=== Processing subject: {subject_id} ===")

        # Define output file for this subject
        tfrecord_filename = tfrecords_output_dir / f"{subject_id}_all_data.tfrecords"

        # Skip if file already exists
        if tfrecord_filename.exists():
            print(f"TFRecord already exists: {tfrecord_filename}. Skipping.")
            continue

        # Process all data for this subject
        processed_data = process_subject_data(subject_id)
        if processed_data is None:
            print(f"Failed to process data for subject {subject_id}. Skipping.")
            continue

        # Write TFRecord file
        try:
            with tf.io.TFRecordWriter(str(tfrecord_filename)) as writer:
                print(f"Writing {len(processed_data)} examples to {tfrecord_filename}")

                evaluation_count = 0
                calibration_count = 0

                for img, eye_type, gaze_coords, is_evaluation, day in processed_data:
                    # Serialize and write the example
                    serialized_example = serialize_example(
                        img, eye_type, gaze_coords, is_evaluation, day, subject_id
                    )
                    writer.write(serialized_example)

                    # Count types
                    if is_evaluation:
                        evaluation_count += 1
                    else:
                        calibration_count += 1

                print(f"✓ Successfully wrote {len(processed_data)} examples")
                print(f"  - {evaluation_count} evaluation images")
                print(f"  - {calibration_count} calibration images")

        except Exception as e:
            print(f"Error writing TFRecord for {subject_id}: {e}")
            # Clean up incomplete file
            if tfrecord_filename.exists():
                tfrecord_filename.unlink()
                print(f"Cleaned up incomplete file: {tfrecord_filename}")

    print("\n✓ Finished creating TFRecords for all subjects")

# Execute the simplified pipeline (comment out to prevent auto-execution)
# create_tfrecords_for_all_subjects()

print("Simplified TFRecords creation pipeline ready. Run create_tfrecords_for_all_subjects() to execute.")

TFRecords output directory: mpiigaze_tfrecords_simplified
Simplified TFRecords creation pipeline ready. Run create_tfrecords_for_all_subjects() to execute.


In [14]:
create_tfrecords_for_all_subjects()

Starting simplified TFRecords creation for all subjects...

=== Processing subject: p00 ===
Loaded 29961 total images for subject p00
  - 2927 evaluation images
  - 27034 calibration images
Subject p00: Found 2927 evaluation images.
Subject p00: Total images to process after filtering: 2927
Processing 2927 images for subject p00...
  Processed 100/2927 images
  Processed 200/2927 images
  Processed 300/2927 images
  Processed 400/2927 images
  Processed 500/2927 images
  Processed 600/2927 images
  Processed 700/2927 images
  Processed 800/2927 images
  Processed 900/2927 images
  Processed 1000/2927 images
  Processed 1100/2927 images
  Processed 1200/2927 images
  Processed 1300/2927 images
  Processed 1400/2927 images
  Processed 1500/2927 images
  Processed 1600/2927 images
  Processed 1700/2927 images
  Processed 1800/2927 images
  Processed 1900/2927 images
  Processed 2000/2927 images
  Processed 2100/2927 images
  Processed 2200/2927 images
  Processed 2300/2927 images
  Proces

## Verify tfrecords

### Subtask:
After writing, add code to read a few examples from the generated TFRecord files using the provided `parse` function (adapted for 'day') to ensure the data is correctly serialized and can be parsed.


**Reasoning**:
Define the parsing function, get a list of TFRecord files, create a dataset, map the parsing function, take a few examples, and iterate through them to print the features for verification.



In [15]:
def parse_tfrecord_example(example_proto):
    """
    Parses a single serialized TFRecord Example with the updated schema
    excluding is_evaluation feature.

    Args:
        example_proto (bytes): A serialized tf.train.Example protocol buffer.

    Returns:
        dict: A dictionary containing the parsed features.
    """
    # Our updated TFRecords schema (excluding is_evaluation)
    feature_description_no_eval = {
        'day': tf.io.FixedLenFeature([], tf.string),
        'subject_id': tf.io.FixedLenFeature([], tf.string),
        'eye_img': tf.io.FixedLenFeature([], tf.string),
        'x': tf.io.FixedLenFeature([], tf.float32),
        'y': tf.io.FixedLenFeature([], tf.float32),
    }

    # Parse the example using the updated feature description
    parsed_features = tf.io.parse_single_example(example_proto, feature_description_no_eval)

    # Decode the eye_img byte string back into a tensor and reshape it
    image_bytes = parsed_features['eye_img']
    image_tensor = tf.io.decode_raw(image_bytes, tf.uint8)
    image_tensor = tf.reshape(image_tensor, (36, 144))
    image_tensor = tf.cast(image_tensor, tf.float32) / 255.0  # Normalize to [0, 1]

    # Replace the raw byte string with the decoded image tensor
    parsed_features['eye_img'] = image_tensor

    # Note: is_evaluation is not in the parsed features

    return parsed_features

def test_parsing_with_sample_data():
    """
    Tests the parsing function with sample data from the simplified TFRecords.
    Assumes TFRecords contain only evaluation images and thus no is_evaluation feature.
    """
    # Look for TFRecord files in the simplified directory
    tfrecords_dir = Path("./mpiigaze_tfrecords_simplified")
    tfrecord_files = sorted(list(tfrecords_dir.glob("*.tfrecords")))

    if not tfrecord_files:
        print(f"No TFRecord files found in {tfrecords_dir}")
        print("Run create_tfrecords_for_all_subjects() first to generate test data.")
        return

    print(f"Found {len(tfrecord_files)} TFRecord files")

    # Test with the first file
    test_file = tfrecord_files[0]
    print(f"Testing parsing with: {test_file}")

    # Create dataset and parse examples
    dataset = tf.data.TFRecordDataset([str(test_file)])
    # Use the updated parsing function
    parsed_dataset = dataset.map(parse_tfrecord_example)

    # Take a few examples for verification
    sample_examples = parsed_dataset.take(5)

    print(f"\nParsing verification - first 5 examples (expecting only evaluation images):")

    # Since we only process evaluation images, all parsed examples should be 'evaluation' conceptually
    evaluation_count = 0
    for i, example in enumerate(sample_examples):
        evaluation_count += 1 # All are evaluation images based on the new processing

        print(f"--- Example {i+1} ---")
        print(f"  Image shape: {example['eye_img'].shape}")
        print(f"  Subject ID: {example['subject_id'].numpy().decode('utf-8')}")
        print(f"  Day: {example['day'].numpy().decode('utf-8')}")
        print(f"  Gaze coords (x, y): ({example['x'].numpy():.4f}, {example['y'].numpy():.4f})")
        # is_evaluation feature is no longer present

    print(f"\nSample breakdown:")
    print(f"  - {evaluation_count} evaluation images (all examples parsed)")


print("Updated parsing function by removing is_evaluation feature.")
print("Run test_parsing_with_sample_data() to verify parsing works correctly with the new schema.")

Updated parsing function by removing is_evaluation feature.
Run test_parsing_with_sample_data() to verify parsing works correctly with the new schema.


## Summary: Simplified MPIIGaze TFRecords Pipeline

### Key Improvements Made:

**1. Simplified Functions (reduced from ~15 to 5 core functions):**
- `crop_black_padding()` - Streamlined black pixel removal with aspect ratio adjustment
- `load_all_subject_data()` - Unified loading of evaluation and calibration images  
- `preprocess_image()` - Combined cropping and resizing
- `process_subject_data()` - End-to-end subject processing
- `create_tfrecords_for_all_subjects()` - Optimized TFRecord generation

**2. Enhanced TFRecords Schema:**
```python
feature_description = {
    'day': tf.string,
    'subject_id': tf.string,
    'eye_img': tf.string,     # 36x144 preprocessed image
    'x': tf.float32,          # Gaze x-coordinate (proportion)
    'y': tf.float32,          # Gaze y-coordinate (proportion)  
    'is_evaluation': tf.bool, # NEW: True=evaluation, False=calibration
}
```

**3. Processing Improvements:**
- **Single pass processing** - No complex day-based grouping
- **One TFRecord per subject** - Easier to manage than per-day files
- **Includes all images** - Both evaluation and calibration data
- **Proper labeling** - `is_evaluation` flag distinguishes image types

**4. Usage:**
```python
# Generate TFRecords for all subjects (evaluation + calibration)
create_tfrecords_for_all_subjects()

# Test parsing
test_parsing_with_sample_data()
```

**5. Output:**
- Files: `{subject_id}_all_data.tfrecords` (15 files total)
- Each file contains all images for that subject with proper `is_evaluation` labeling
- Maintains same image preprocessing (black pixel removal + 36x144 resize)
- Preserves gaze coordinate accuracy

### Next Steps:
1. Run `create_tfrecords_for_all_subjects()` to generate the simplified dataset
2. Use `test_parsing_with_sample_data()` to verify the pipeline works correctly
3. The resulting TFRecords can be used for training with both evaluation and calibration data properly labeled

In [16]:
## Model Evaluation: Updated TFRecords Parsing

def parse_mpiigaze_tfrecords(element):
    """
    Parse function adapted for our new MPIIGaze TFRecords schema (no is_evaluation).
    Maps our schema to the format expected by the individual model analysis.
    Since only evaluation images are included, we will assume phase=2 (evaluation).
    """
    # Our TFRecords schema (excluding is_evaluation)
    data_structure = {
        'day': tf.io.FixedLenFeature([], tf.string),
        'subject_id': tf.io.FixedLenFeature([], tf.string),
        'eye_img': tf.io.FixedLenFeature([], tf.string),
        'x': tf.io.FixedLenFeature([], tf.float32),
        'y': tf.io.FixedLenFeature([], tf.float32),
        # Removed 'is_evaluation' from data_structure
    }

    content = tf.io.parse_single_example(element, data_structure)

    # Extract data
    raw_image = content['eye_img']
    x_coord = content['x']
    y_coord = content['y']
    subject_id_str = content['subject_id']

    # Since only evaluation images are included, we set phase=2 (evaluation)
    phase = tf.constant(2, dtype=tf.int64)

    # Parse image from raw bytes
    image = tf.io.decode_raw(raw_image, tf.uint8)
    image = tf.reshape(image, (36, 144))  # Our images are already 36x144
    image = tf.expand_dims(image, -1)  # Add channel dimension

    # Convert string subject_id to int64 for compatibility
    # Create a hash of the subject_id string to get consistent integer IDs
    subject_id_hash = tf.strings.to_hash_bucket_fast(subject_id_str, 2**31-1)
    subject_id = tf.cast(subject_id_hash, tf.int64)

    # Coordinates are already in 0-1 range, no scaling needed
    coords = [x_coord, y_coord]

    return image, phase, coords, subject_id

print("Updated parsing function for MPIIGaze TFRecords by removing is_evaluation feature.")
print("Phase is now hardcoded to 2 (evaluation) as only evaluation images are processed.")

Updated parsing function for MPIIGaze TFRecords by removing is_evaluation feature.
Phase is now hardcoded to 2 (evaluation) as only evaluation images are processed.


In [17]:
## Model Loading: Custom Layers and Architecture

# Fixed constants from the original analysis
MAX_TARGETS = 3020
EMBEDDING_DIM = 200  # From the individual model run config
RIDGE_REGULARIZATION = 0.001  # From the individual model run config
BACKBONE = "densenet"  # From the individual model run config

class SimpleTimeDistributed(keras.layers.Wrapper):
    def __init__(self, layer, **kwargs):
        super().__init__(layer, **kwargs)
        self.supports_masking = getattr(layer, 'supports_masking', False)

    def build(self, input_shape):
        if not isinstance(input_shape, (tuple, list)) or len(input_shape) < 3:
            raise ValueError(
                "`SimpleTimeDistributed` requires input with at least 3 dimensions"
            )
        super().build((input_shape[0], *input_shape[2:]))
        self.built = True

    def compute_output_shape(self, input_shape):
        child_output_shape = self.layer.compute_output_shape((input_shape[0], *input_shape[2:]))
        return (child_output_shape[0], input_shape[1], *child_output_shape[1:])

    def call(self, inputs, training=None):
        input_shape = ops.shape(inputs)
        batch_size = input_shape[0]
        time_steps = input_shape[1]

        reshaped_inputs = ops.reshape(inputs, (-1, *input_shape[2:]))
        outputs = self.layer.call(reshaped_inputs, training=training)
        output_shape = ops.shape(outputs)

        return ops.reshape(outputs, (batch_size, time_steps, *output_shape[1:]))

class MaskedWeightedRidgeRegressionLayer(keras.layers.Layer):
    def __init__(self, lambda_ridge, epsilon=1e-6, **kwargs):
        self.lambda_ridge = lambda_ridge
        self.epsilon = epsilon
        super(MaskedWeightedRidgeRegressionLayer, self).__init__(**kwargs)

    @tf.function
    def call(self, inputs):
        embeddings, coords, calibration_weights, cal_mask = inputs

        embeddings = ops.cast(embeddings, "float32")
        coords = ops.cast(coords, "float32")
        calibration_weights = ops.cast(calibration_weights, "float32")
        cal_mask = ops.cast(cal_mask, "float32")

        w = ops.squeeze(calibration_weights, axis=-1)
        w_masked = w * cal_mask
        w_sqrt = ops.sqrt(w_masked + self.epsilon)
        w_sqrt = ops.expand_dims(w_sqrt, -1)

        cal_mask_expand = ops.expand_dims(cal_mask, -1)
        X = embeddings * cal_mask_expand
        X_weighted = X * w_sqrt
        y_weighted = coords * w_sqrt * cal_mask_expand

        X_t = ops.transpose(X_weighted, axes=[0, 2, 1])
        X_t_X = ops.matmul(X_t, X_weighted)

        identity_matrix = ops.cast(ops.eye(ops.shape(embeddings)[-1]), "float32")
        lhs = X_t_X + self.lambda_ridge * identity_matrix
        rhs = ops.matmul(X_t, y_weighted)

        kernel = tf.linalg.solve(lhs, rhs)
        output = ops.matmul(embeddings, kernel)

        return output

    def compute_output_shape(self, input_shapes):
        unknown_embeddings_shape, _, _, _ = input_shapes
        return unknown_embeddings_shape[:-1] + (2,)

    def get_config(self):
        config = super(MaskedWeightedRidgeRegressionLayer, self).get_config()
        config.update({"lambda_ridge": self.lambda_ridge, "epsilon": self.epsilon})
        return config

def normalized_weighted_euc_dist(y_true, y_pred):
    """Normalized weighted euclidean distance metric from original analysis."""
    x_weight = ops.convert_to_tensor([1.778, 1.0], dtype="float32")

    y_true_weighted = ops.multiply(x_weight, y_true)
    y_pred_weighted = ops.multiply(x_weight, y_pred)

    squared_diff = ops.square(y_pred_weighted - y_true_weighted)
    squared_dist = ops.sum(squared_diff, axis=-1)
    dist = ops.sqrt(squared_dist)

    normalized_dist = ops.divide(dist, .0203992)
    return normalized_dist

print("Custom layers and loss function loaded successfully.")

Custom layers and loss function loaded successfully.


In [18]:
## Model Architecture and Loading

import keras_hub

def create_dense_net_backbone():
    DENSE_NET_STACKWISE_NUM_REPEATS = [4,4,4]
    return keras_hub.models.DenseNetBackbone(
        stackwise_num_repeats=DENSE_NET_STACKWISE_NUM_REPEATS,
        image_shape=(36, 144, 1),
    )

def create_embedding_model(backbone_type):
    image_shape = (36, 144, 1)
    input_eyes = keras.layers.Input(shape=image_shape)
    eyes_rescaled = keras.layers.Rescaling(scale=1./255)(input_eyes)

    if backbone_type == "densenet":
        backbone = create_dense_net_backbone()

    backbone_encoder = backbone(eyes_rescaled)
    flatten_compress = keras.layers.Flatten()(backbone_encoder)
    eye_embedding = keras.layers.Dense(units=EMBEDDING_DIM, activation="tanh")(flatten_compress)

    embedding_model = keras.Model(inputs=input_eyes, outputs=eye_embedding, name="Eye_Image_Embedding")
    return embedding_model

def create_masked_model():
    input_all_images = keras.layers.Input(
        shape=(MAX_TARGETS, 36, 144, 1),
        name="Input_All_Images"
    )

    input_all_coords = keras.layers.Input(
        shape=(MAX_TARGETS, 2),
        name="Input_All_Coords"
    )

    input_cal_mask = keras.layers.Input(
        shape=(MAX_TARGETS,),
        name="Input_Calibration_Mask",
    )

    embedding_model = create_embedding_model(BACKBONE)
    all_embeddings = SimpleTimeDistributed(embedding_model, name="Image_Embeddings")(input_all_images)

    calibration_weights = keras.layers.Dense(
        1,
        activation="sigmoid",
        name="Calibration_Weights"
    )(all_embeddings)

    ridge = MaskedWeightedRidgeRegressionLayer(
        RIDGE_REGULARIZATION,
        name="Regression"
    )(
        [
            all_embeddings,
            input_all_coords,
            calibration_weights,
            input_cal_mask,
        ],
    )

    full_model = keras.Model(
        inputs=[
            input_all_images,
            input_all_coords,
            input_cal_mask,
        ],
        outputs=ridge,
        name="MaskedEyePredictionModel"
    )

    return full_model

# Create and compile the model
print("Creating model architecture...")
evaluation_model = create_masked_model()
evaluation_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=normalized_weighted_euc_dist,
    metrics=[normalized_weighted_euc_dist],
    jit_compile=False
)

print("Model created successfully. Summary:")
evaluation_model.summary()

Creating model architecture...
Model created successfully. Summary:


Model: "MaskedEyePredictionModel"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input_All_Images    │ (None, 3020, 36,  │          0 │ -                 │
│ (InputLayer)        │ 144, 1)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Image_Embeddings    │ (None, 3020, 200) │  1,581,896 │ Input_All_Images… │
│ (SimpleTimeDistrib… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Input_All_Coords    │ (None, 3020, 2)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Calibration_Weights │ (None, 3020, 1)   │        201 │ Image_Embeddings… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Input_Calibration_… │ (None, 3020)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Regression          │ (None, 3020, 2)   │          0 │ Image_Embeddings… │
│ (MaskedWeightedRid… │                   │            │ Input_All_Coords… │
│                     │                   │            │ Calibration_Weig… │
│                     │                   │            │ Input_Calibratio… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,582,097 (6.04 MB)

 Trainable params: 1,574,257 (6.01 MB)

 Non-trainable params: 7,840 (30.62 KB)

In [19]:
from google.colab import userdata
import os

# Import the W&B API key from Colab secrets and set it as an environment variable
WANDB_API_KEY = userdata.get('WANDB_API_KEY')
os.environ["WANDB_API_KEY"] = WANDB_API_KEY

print("WANDB_API_KEY imported from Colab secrets and set as environment variable.")

WANDB_API_KEY imported from Colab secrets and set as environment variable.


In [20]:
## Load Pre-trained Model Weights

import wandb

# Note: This section requires W&B credentials to download the model weights
# For demonstration purposes, we'll show the code structure

def load_pretrained_weights():
    """
    Load pre-trained model weights from W&B.
    This requires proper W&B authentication and project access.
    """
    try:
        # W&B configuration from individual model run
        EXPERIMENT_ID = "gf1zjs9v"
        ENTITY_NAME = "vassar-cogsci-lab"
        PROJECT_NAME = "eye-tracking-dense-full-data-set-single-eye"

        # Initialize W&B and download weights
        print("Initializing W&B connection...")
        wandb.login()  # Requires API key
        api = wandb.Api()
        run = api.run(f"{ENTITY_NAME}/{PROJECT_NAME}/{EXPERIMENT_ID}")
        run.file("full_model.weights.h5").download(exist_ok=True)

        # Load weights into model
        evaluation_model.load_weights('full_model.weights.h5')
        print("✓ Pre-trained model weights loaded successfully!")

        return True

    except Exception as e:
        print(f"❌ Error loading pre-trained weights: {e}")
        print("   Continuing with randomly initialized weights")
        return False

# Attempt to load pre-trained weights
weights_loaded = load_pretrained_weights()

print(f"Model ready for evaluation (pre-trained weights: {'✓' if weights_loaded else '✗'})")

Initializing W&B connection...


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


✓ Pre-trained model weights loaded successfully!
Model ready for evaluation (pre-trained weights: ✓)


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 175 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [28]:
## Create Evaluation Dataset from Simplified TFRecords

# MAX_TARGETS represents the maximum number of images per subject in a batch
# This should be large enough to accommodate the subject with the most evaluation images.
# Let's set it to a safe value, e.g., 3020, considering the max evaluation images are around 3000.
MAX_TARGETS = 3020

def load_mpiigaze_dataset():
    """
    Load our simplified TFRecords dataset and process it for model evaluation.
    """
    # Look for our TFRecord files
    tfrecords_dir = Path("./mpiigaze_tfrecords_simplified")
    if not tfrecords_dir.exists():
        print(f"❌ TFRecords directory not found: {tfrecords_dir}")
        print("   Please run create_tfrecords_for_all_subjects() first to generate the dataset.")
        return None

    # Get all TFRecord files
    tfrecord_files = sorted(list(tfrecords_dir.glob("*.tfrecords")))
    if not tfrecord_files:
        print(f"❌ No TFRecord files found in {tfrecords_dir}")
        return None

    print(f"Found {len(tfrecord_files)} TFRecord files")

    # Create dataset from all files
    dataset = tf.data.TFRecordDataset([str(f) for f in tfrecord_files])

    # Parse the records using our updated parsing function (which now assumes evaluation images)
    parsed_dataset = dataset.map(parse_mpiigaze_tfrecords, num_parallel_calls=tf.data.AUTOTUNE)

    print("✓ Dataset loaded and parsed successfully")
    return parsed_dataset


def create_dataset_with_random_masks(dataset, num_calibration_points=20):
    """
    Creates evaluation dataset from a source dataset (containing only evaluation images)
    by generating random calibration and target masks for each subject's batch.
    """
    def group_subject_data(subject_id, ds):
        # Batch all of a subject's data together
        return ds.batch(batch_size=MAX_TARGETS)


    grouped_dataset = dataset.group_by_window(
        key_func=lambda img, phase, coords, subject_id: subject_id,
        reduce_func=group_subject_data,
        window_size=MAX_TARGETS * 2 # Use a larger window size to group all images for a subject
    )


    def generate_masks_and_pad(images, phase, coords, subject_ids):
        actual_batch_size = tf.shape(images)[0]

        # Ensure there are enough images for calibration
        # If not, use all available for calibration (this is a fallback and might not be ideal)
        effective_num_calibration = tf.minimum(num_calibration_points, actual_batch_size)

        # Generate random permutation of indices
        permutation = tf.random.shuffle(tf.range(actual_batch_size))

        # Select indices for calibration and target sets
        cal_indices = tf.sort(permutation[:effective_num_calibration])
        target_indices = tf.sort(permutation[effective_num_calibration:])

        # Create calibration mask
        cal_mask = tf.scatter_nd(
            tf.expand_dims(cal_indices, axis=1),
            tf.ones(tf.shape(cal_indices)[0], dtype=tf.int8),
            shape=[actual_batch_size]
        )

        # Create target mask
        target_mask = tf.scatter_nd(
            tf.expand_dims(target_indices, axis=1),
            tf.ones(tf.shape(target_indices)[0], dtype=tf.int8),
            shape=[actual_batch_size]
        )

        # Pad data and masks to fixed size
        padded_images = tf.pad(
            tf.reshape(images, (-1, 36, 144, 1)),
            [[0, MAX_TARGETS - actual_batch_size], [0, 0], [0, 0], [0, 0]]
        )
        padded_coords = tf.pad(
            coords,
            [[0, MAX_TARGETS - actual_batch_size], [0, 0]]
        )
        padded_cal_mask = tf.pad(
            cal_mask,
            [[0, MAX_TARGETS - actual_batch_size]]
        )
        padded_target_mask = tf.pad(
            target_mask,
            [[0, MAX_TARGETS - actual_batch_size]]
        )

        # Ensure shapes
        padded_images = tf.ensure_shape(padded_images, [MAX_TARGETS, 36, 144, 1])
        padded_coords = tf.ensure_shape(padded_coords, [MAX_TARGETS, 2])
        padded_cal_mask = tf.ensure_shape(padded_cal_mask, [MAX_TARGETS])
        padded_target_mask = tf.ensure_shape(padded_target_mask, [MAX_TARGETS])

        # Return features tuple, labels, target_mask, and subject_ids
        return (padded_images, padded_coords, padded_cal_mask, padded_target_mask), padded_coords, padded_target_mask, subject_ids[0]


    masked_dataset = grouped_dataset.map(
        generate_masks_and_pad,
        num_parallel_calls=tf.data.AUTOTUNE
    )

    return masked_dataset

def prepare_model_inputs_for_evaluation(features, labels, target_mask, subject_ids):
    """Prepare inputs in the format expected by the model."""
    images, coords, cal_mask, target_mask_from_features = features # Unpack features tuple

    # Explicitly ensure image shape and type for model input
    images = tf.ensure_shape(images, [MAX_TARGETS, 36, 144, 1])
    images = tf.cast(images, tf.float32) # Cast to float32
    # Note: Normalization 0-1 is already done in parse_tfrecord_example


    inputs = {
        "Input_All_Images": images,
        "Input_All_Coords": coords,
        "Input_Calibration_Mask": cal_mask,
    }

    return inputs, labels, target_mask, subject_ids


# Load the dataset
print("Loading MPIIGaze dataset...")
mpiigaze_dataset = load_mpiigaze_dataset()

if mpiigaze_dataset is not None:
    print("Creating evaluation dataset with random calibration and target masks per subject...")

    # Create masked dataset using the random mask generation logic
    evaluation_masked_dataset = create_dataset_with_random_masks(mpiigaze_dataset, num_calibration_points=20)

    # Prepare for model input
    evaluation_dataset = evaluation_masked_dataset.map(
        lambda features_tuple, labels, target_mask, subject_id: prepare_model_inputs_for_evaluation(features_tuple, labels, target_mask, subject_id),
        num_parallel_calls=tf.data.AUTOTUNE
    ).prefetch(tf.data.AUTOTUNE)

    # Count subjects
    subject_count = 0
    print("Counting subjects in evaluation dataset...")
    for _ in evaluation_dataset:
        subject_count += 1

    print(f"✓ Evaluation dataset ready with {subject_count} subjects")
else:
    print("❌ Failed to load dataset")

Loading MPIIGaze dataset...
Found 15 TFRecord files
✓ Dataset loaded and parsed successfully
Creating evaluation dataset with random calibration and target masks per subject...
Counting subjects in evaluation dataset...
✓ Evaluation dataset ready with 15 subjects


In [29]:
evaluation_dataset.element_spec

({'Input_All_Images': TensorSpec(shape=(3020, 36, 144, 1), dtype=tf.float32, name=None),
  'Input_All_Coords': TensorSpec(shape=(3020, 2), dtype=tf.float32, name=None),
  'Input_Calibration_Mask': TensorSpec(shape=(3020,), dtype=tf.int8, name=None)},
 TensorSpec(shape=(3020, 2), dtype=tf.float32, name=None),
 TensorSpec(shape=(3020,), dtype=tf.int8, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [26]:
evaluation_model.input_shape

[(None, 3020, 36, 144, 1), (None, 3020, 2), (None, 3020)]

In [ ]:
## Run Model Evaluation and Calculate Results

def run_mpiigaze_evaluation():
    """
    Run the model evaluation on MPIIGaze dataset and calculate normalized euclidean distance.
    """
    if 'evaluation_dataset' not in locals() and 'evaluation_dataset' not in globals():
        print("❌ Evaluation dataset not ready. Please run the dataset creation cell first.")
        return None

    print("Running model evaluation on MPIIGaze dataset...")
    print("This may take several minutes depending on dataset size...")

    # Generate predictions for the entire dataset
    print("Generating predictions for all subjects...")
    # evaluation_dataset yields (inputs, labels, target_mask, subject_id)
    # We only need to pass the inputs dictionary to predict
    predictions = evaluation_model.predict(
        evaluation_dataset.map(lambda inputs, labels, target_mask, subject_id: inputs).batch(1),
        verbose=1 # Keep verbose=1 to see prediction progress
    )

    print("\nProcessing results and calculating errors per subject...")
    all_results = []

    # Iterate through the dataset again to match predictions with ground truth and masks
    # We use evaluation_dataset directly as it yields one subject's data per element
    for i, (inputs, labels, target_mask, subject_id) in enumerate(evaluation_dataset):

        # Extract subject info (subject_id is already a scalar tensor)
        current_subject_id = int(subject_id.numpy())

        # Get the corresponding predictions for this subject
        # predictions is a single array, the i-th element corresponds to the i-th subject's batch
        pred_batch = predictions[i]

        # Get actual coordinates and target mask
        actual_coords = labels.numpy()  # Labels is already the batch of coords for this subject
        mask = target_mask.numpy().astype(bool)  # Target mask for this subject

        # Filter by mask to get only evaluation points
        pred_coords = pred_batch[mask]
        actual_coords_filtered = actual_coords[mask]

        # Calculate errors for each prediction
        individual_errors = []
        if actual_coords_filtered.shape[0] > 0: # Ensure there are evaluation points
            for pred_coord, actual_coord in zip(pred_coords, actual_coords_filtered):
                # Calculate error using the defined metric
                # Need to reshape to (1, 2) as normalized_weighted_euc_dist expects batch dimension
                error = normalized_weighted_euc_dist(
                    np.array([actual_coord], dtype=np.float32),
                    np.array([pred_coord], dtype=np.float32)
                )
                individual_errors.append(float(error.numpy()[0]))

            # Calculate mean error for this subject
            mean_error = np.mean(individual_errors)
            all_results.append({
                'subject_id': current_subject_id,
                'mean_error': mean_error,
                'n_evaluation_points': len(individual_errors),
                'individual_errors': individual_errors # Store individual errors for potential further analysis
            })
        else:
            print(f"  Subject {current_subject_id} has no evaluation points in the target mask. Skipping error calculation.")


        if (i + 1) % 5 == 0: # Print progress every 5 subjects
            print(f"  Processed results for {i + 1} subjects...")

    print("\nFinished processing results and calculating errors.")
    return all_results

def analyze_results(results):
    """
    Analyze the evaluation results and generate summary statistics.
    """
    if not results:
        print("❌ No results to analyze")
        return None

    print(f"\n=== MPIIGaze Model Evaluation Results ===")
    print(f"Total subjects evaluated: {len(results)}")

    # Extract mean errors and evaluation points from subjects with results
    mean_errors = [r['mean_error'] for r in results]
    n_evaluation_points = [r['n_evaluation_points'] for r in results]
    subject_ids_with_results = [r['subject_id'] for r in results]

    # Calculate summary statistics
    overall_mean = np.mean(mean_errors)
    overall_std = np.std(mean_errors)
    overall_median = np.median(mean_errors)

    print(f"\n--- Summary Statistics ---")
    print(f"Overall mean error: {overall_mean:.4f} (normalized euclidean distance)")
    print(f"Standard deviation: {overall_std:.4f}")
    print(f"Median error: {overall_median:.4f}")
    print(f"Min error: {np.min(mean_errors):.4f}")
    print(f"Max error: {np.max(mean_errors):.4f}")

    print(f"\n--- Evaluation Points per Subject (for subjects with results) ---")
    print(f"Mean evaluation points: {np.mean(n_evaluation_points):.1f}")
    print(f"Min evaluation points: {np.min(n_evaluation_points)}")
    print(f"Max evaluation points: {np.max(n_evaluation_points)}")

    # Create results DataFrame
    results_df = pd.DataFrame({
        'subject_id': subject_ids_with_results,
        'mean_error': mean_errors,
        'n_evaluation_points': n_evaluation_points
    })

    return results_df, {
        'overall_mean': overall_mean,
        'overall_std': overall_std,
        'overall_median': overall_median,
        'n_subjects': len(results)
    }

# Run the evaluation (this will be slow without pre-trained weights)
print("⚠️  Note: This evaluation will use randomly initialized weights since W&B credentials are not configured.")
print("    With proper pre-trained weights, the results would be meaningful.")

# Uncomment the following lines to run the actual evaluation:
evaluation_results = run_mpiigaze_evaluation()

if evaluation_results:
    results_df, summary_stats = analyze_results(evaluation_results)
    print("\n✓ Evaluation completed successfully!")
else:
    print("\n❌ Evaluation failed")

print("\n💡 To run the evaluation:")
print("   1. Configure W&B credentials to load pre-trained weights")
print("   2. Ensure the simplified TFRecords dataset is available")
print("   3. Uncomment the evaluation code above")
print("   4. Run this cell")

⚠️  Note: This evaluation will use randomly initialized weights since W&B credentials are not configured.
    With proper pre-trained weights, the results would be meaningful.
Running model evaluation on MPIIGaze dataset...
This may take several minutes depending on dataset size...
Generating predictions for all subjects...
      7/Unknown 141s 19s/step

## Final Summary: Complete MPIIGaze Evaluation Pipeline

### What We've Built

This notebook now provides a complete pipeline for MPIIGaze evaluation:

**1. Simplified TFRecords Creation:**
- ✅ Streamlined data loading (5 core functions vs. original 15+)
- ✅ Includes both evaluation and calibration images
- ✅ Enhanced schema with `is_evaluation` boolean feature
- ✅ Maintains black pixel removal with 36x144 output
- ✅ One TFRecord per subject for easier management

**2. Model Evaluation Integration:**
- ✅ Compatible parsing function for new TFRecords schema
- ✅ Pre-trained model architecture (DenseNet + masked ridge regression)
- ✅ Evaluation methodology matching original analysis
- ✅ 20 random calibration images per subject + all evaluation images
- ✅ Normalized euclidean distance metric calculation

**3. Complete Evaluation Workflow:**
```python
# 1. Generate simplified TFRecords dataset
create_tfrecords_for_all_subjects()

# 2. Load and evaluate with pre-trained model
mpiigaze_dataset = load_mpiigaze_dataset()
evaluation_dataset = create_calibration_subset_with_20_points(mpiigaze_dataset)
evaluation_results = run_mpiigaze_evaluation()

# 3. Analyze results
results_df, summary_stats = analyze_results(evaluation_results)
```

**4. Key Features:**
- **Maintains scientific rigor**: Same evaluation protocol as original study
- **Includes calibration data**: All non-evaluation images labeled appropriately
- **Simplified codebase**: Much cleaner and more maintainable
- **Flexible evaluation**: Can easily modify number of calibration points
- **Comprehensive analysis**: Per-subject and overall error metrics

### Next Steps for Full Evaluation:

1. **Configure W&B access** to download pre-trained model weights
2. **Run TFRecords generation**: Execute `create_tfrecords_for_all_subjects()`
3. **Execute evaluation**: Uncomment and run the evaluation code
4. **Compare results**: Analyze performance against original benchmarks

The pipeline is now ready for MPIIGaze evaluation with both evaluation and calibration images properly integrated.